# Assignment 5


In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import numpy as np

## Question 1 
### a)
Any single error can be considered as the insertion of a Pauli $X,Y,Z$ gate into the quantum circuit of the Steane code. There are eight operations in the Cat state preparation and verification which leads to 9 positions over the seven qubits.

#### Verification ancilla
Starting with the 3 verification ancilla.
Any $X$ error on these qubits will just propagate to the output measurement of the ancilla and will not effect the cat state itself. This will cause a single error on the same qubit as the error occurred.
Any $Z$ error on the same 3 qubits will propagate onto two qubits of the cat state through the CNOT's. This will not effect the verification measurement and the two $Z$ errors will cancel at the decoding CNOT causing no error to occur. If a $Z$ error occurs between two CNOT's on the same qubit a $Z$ error will propagate onto a single qubit of the cat state. This will cause a single $X$ error on the cat state measurement output on the top line. Therefore any $Z$ error on the verification ancilla will cause at most one error on the ancilla output.

#### Cat state preparation
An $X$ error on the start of first qubit will cause an $X$ error on the output as it will not spread to any other lines. $X$ errors on the input of qubits $2-4$ will propagate to the verification measurement causing the process to fail. It will not however propagate to the cat state output. A $Z$ at the start of the first qubit will cause no error on the verification output or cat state output. $Z$ error at the input of qubits $2-4$ will cause no errors to occur. A $X$ or $Z$ error after the Hadamard gate will just change the opposite and will cause the errors as described before.

### Cat state verification
A $Z$ error along the first qubit will cause an $X$ error at the cat state output. Any $X$ error on qubits $2-4$ of the cat state verification will cause the verification ancilla to fail but will not propagate into the cat state output. Any $Z$ on qubits $2-4$ of the cat state verification will cause an $X$ error on the cat state output but not the verification ancilla output. 



### b) 
As shown in Verification ancilla section a $Z$ error propagate up to the cat state qubits but then will not propagate down into the encoded data as the $Z$ error will pass over the control of the controlled-M measurement. It will however than propagate up into the first qubit and cause an $X$ error.

### c)
This covers all possible sources of error as $Y$ can be considered as $iXZ$ errors. Since $X$ and $Z$ errors at most caused a single error on the ancilla output and no $X$ and $Z$ error at the same position caused an error. It can be considered that no error will lead to more than on error on the output.

## Question 2
### a)
By inspection the stabiliser generators are:
$$
\begin{aligned}
G_1 &= X_1X_2X_3X_4\\
G_2 &= Z_1Z_2Z_3Z_4
\end{aligned}
$$

### b)
Again inspecting the code words:
$$
\begin{aligned}
\ket{10}_L &= X_{L_1}\ket{00}\\
\ket{11}_L &= X_{L_1}\ket{01}\\
\ket{00}_L &= X_{L_1}\ket{10}\\
\ket{01}_L &= X_{L_1}\ket{11}\\
X_{L_1} &= X_1I_2X_3I_4
\end{aligned}
$$
Similarly:
$$
\begin{aligned}
X_{L_2} &= X_1X_2I_3I_4\\
Z_{L_1} &= I_1I_2Z_3Z_4\\
Z_{L_2} &= I_1Z_2I_3Z_4
\end{aligned}
$$

We test if these commute with the stabiliser generators:
$$
\begin{aligned}
    [X_{L_1},G_1] &= I_1X_2I_3X_4 - I_1X_2I_3X_4 = 0\\
    [X_{L_1},G_2] &= iY_1Z_2iY_3Z_4 - -iY_1Z_2-iY_3Z_4 = 0\\
    [X_{L_2},G_1] &= I_1I_2X_3X_4 - I_1I_2X_3X_4 = 0\\
    [X_{L_2},G_2] &= iY_1iY_2Z_3Z_4 -  -iY_1-iY_2Z_3Z_4 = 0\\
    [Z_{L_1},G_1] &= X_1X_2-iY_3-iY_4 - X_1X_2iY_3iY_4 = 0\\
    [Z_{L_1},G_2] &= Z_1Z_2I_3I_4 - Z_1Z_2I_3I_4 = 0\\
    [Z_{L_2},G_1] &= X_1-iY_2X_3-iY_4 - X_1iY_2X_3iY_4 = 0\\
    [Z_{L_2},G_2] &= Z_1I_2Z_3I_4 - Z_1I_2Z_3I_4 = 0
\end{aligned}
$$

and that they anti-commutate with eac other:

$$
\begin{aligned}
\{X_{L_1},Z_{L_1}\} &= X_1I_2iY_3Z_4 + X_1I_2-iY_3Z_4 = 0\\
\{X_{L_2},Z_{L_2}\} &= X_1iY_2I_3Z_4 + X_1-iY_2I_3Z_4 = 0\\
\end{aligned}
$$


### c)
The circuit for detecting an error on the $\left[\left[ 4,2\right]\right]$ encoded data can be implemented through measurement of its stabiliser generators is created by adding 2 ancilla corresponding to the 2 stabiliser generators are prepared in the computational basis. Controlled stabiliser measurements are applied applied to the state. This gives the following circuit created in qiskit.

In [2]:
q = QuantumRegister(4,name="q")
a = QuantumRegister(2,name="anc")
out = ClassicalRegister(2,name="anc_out")

error_detect_circuit = QuantumCircuit(q,a,out)
error_detect_circuit.h(a)

# stabiliser X_1X_2X_3X_4
error_detect_circuit.cnot(a[0],q)
error_detect_circuit.cz(a[1],q)

error_detect_circuit.measure(a, out)

error_detect_circuit.draw()

┌───┐                                 
      q_0: ─────┤ X ├──────■──────────────────────────
                └─┬─┘┌───┐ │                          
      q_1: ───────┼──┤ X ├─┼───────■──────────────────
                  │  └─┬─┘ │ ┌───┐ │                  
      q_2: ───────┼────┼───┼─┤ X ├─┼───────■──────────
                  │    │   │ └─┬─┘ │ ┌───┐ │          
      q_3: ───────┼────┼───┼───┼───┼─┤ X ├─┼──■───────
           ┌───┐  │    │   │   │   │ └─┬─┘ │  │ ┌─┐   
    anc_0: ┤ H ├──■────■───┼───■───┼───■───┼──┼─┤M├───
           ├───┤           │       │       │  │ └╥┘┌─┐
    anc_1: ┤ H ├───────────■───────■───────■──■──╫─┤M├
           └───┘                                 ║ └╥┘
anc_out: 2/══════════════════════════════════════╩══╩═
                                                 0  1

### d)
The code distance of the [[4,2]] error detection code is $d=2$. In order to correct errors the code distance must $d \ge 3$.

## Question 3

### a)
The encoded state $\ket{\psi}$ is expanded to be the following:

$$
\ket{\psi} = \frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1100} + \ket{0011} \right)
$$

The erasure error is caused by doing partial trace over the Hilbert space of the qubit erased for the density operator of the system. For a pure state this operation can be trivially simplified to removing the qubit that is erased. Thus the state of the system suffering an erasure on the 1st qubit becomes $\ket{\psi}_{e}$:
$$
\begin{aligned}
\ket{\psi}_e &= \mathrm{tr}_{\mathcal{H_1}}\left(\ket{\psi}\right)\\
    &= \frac{\alpha}{\sqrt{2}}\left(\ket{000} + \ket{111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{100} + \ket{011} \right)
\end{aligned}
$$
The replacement of the qubit is done by tensoring $\ket{0}$ with the erased state to give the recovery state $\ket{\psi}_r$:
$$
\begin{aligned}
\ket{\psi}_r &= \ket{0} \otimes \ket{\psi}_e\\
    &= \frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{0111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{0100} + \ket{0011} \right)
\end{aligned}
$$
**_NOTE:_** All of the above holds for any pure state $\ket{\psi}$ and can be proved with density operator formulism.

We create a set of projection operator in the basis states $\ket{0},\ket{1}$ of the second qubit shown below:
$$
\begin{aligned}
P_0 &= I\otimes \ket{0}\bra{0} \otimes I \otimes I\\
P_1 &= I\otimes \ket{1}\bra{1} \otimes I \otimes I
\end{aligned}
$$
#### Projection $P_0$
Using the probability the state must be normalised and thus $\alpha^2 + \beta^2 = 1$, the probability of each outcome occurring is $0.5$. Therefore the resulting states after the projective measurement can calculated. If the first outcome occurs $\ket{\psi}_r$ becomes:
$$
\begin{aligned}
\ket{\psi}_r &= \frac{P_0 \ket{\psi}_r}{\sqrt{\bra{\psi}_r P_0 \ket{\psi}_r}}\\
\ket{\psi}_r &= \frac{P_0 \ket{\psi}_r}{\sqrt{p_0}}\\
&= \frac{\alpha}{2}\ket{0000} + \frac{\beta}{2}\ket{0011}
\end{aligned}
$$
The state can be corrected by applying the following circuit:


$$
\begin{aligned}
\ket{\psi}_r &= H_1I_2I_3I_4\ket{\psi}_r \\
    &=  \frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1000} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1011} + \ket{0011} \right)\\
\ket{\psi}_r &= \mathrm{CNOT}_{1,(2,3,4)} \ket{\psi}_r\\
    &= \mathrm{CNOT}_{1,(2,3)}I_4 \left(\frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1001} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1010} + \ket{0011} \right)\right)\\
    &= \mathrm{CNOT}_{1,2}I_3I_4 \left(\frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1011} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1000} + \ket{0011} \right)\right)\\
    &= \frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1100} + \ket{0011} \right)\\
    &= \ket{\psi}
\end{aligned}
$$
#### Projection $P_1$
If the second outcome caused by the projector $P_1$ occurs $\ket{\psi}_r$ becomes:
$$
\begin{aligned}
\ket{\psi}_r &= \frac{P_1 \ket{\psi}_r}{\sqrt{\bra{\psi}_r P_1 \ket{\psi}_r}}\\
\ket{\psi}_r &= \frac{P_1 \ket{\psi}_r}{\sqrt{p_1}}\\
&= \frac{\alpha}{2}\ket{0111} + \frac{\beta}{2}\ket{0100}
\end{aligned}
$$
If this is the case the state can be correct with the following circuit identical to the first with an additional $X$ gate on the erased qubit.

$$
\begin{aligned}
\ket{\psi}_r &= H_1I_2I_3I_4\ket{\psi}_r \\
    &=  \frac{\alpha}{\sqrt{2}}\left(\ket{0111} + \ket{1111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{0100} + \ket{1100} \right)\\
\ket{\psi}_r &= \mathrm{CNOT}_{1,(2,3,4)} \ket{\psi}_r\\
    &= \mathrm{CNOT}_{1,(2,3)}I_4 \left(\frac{\alpha}{\sqrt{2}}\left(\ket{0111} + \ket{1110} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{0100} + \ket{1101} \right)\right)\\
    &= \mathrm{CNOT}_{1,2}I_3I_4 \left(\frac{\alpha}{\sqrt{2}}\left(\ket{0111} + \ket{1100} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{0100} + \ket{1111} \right)\right)\\
    &= \frac{\alpha}{\sqrt{2}}\left(\ket{0111} + \ket{1000} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{0100} + \ket{1011} \right)\\
\ket{\psi}_r &= X_1I_2I_3I_4\ket{\psi}_r\\
    &= \frac{\alpha}{\sqrt{2}}\left(\ket{0000} + \ket{1111} \right) + \frac{\beta}{\sqrt{2}}\left(\ket{1100} + \ket{0011} \right)\\
    &= \ket{\psi}
\end{aligned}
$$

### b)
The generalised form of the erasure correction can be created through the procedure outlined in a) but applied to an arbitrary qubit erasure. For an erasure on the $n\mathrm{th}$ qubit a projective measurement in the computational basis $\ket{0},\ket{1}$ is performed on the $m\mathrm{th}$ qubit given by $m = n-1+2(n\mod2)$. The $m\mathrm{th}$ qubit can be considered the other qubit of each pair grouping in 2's ie; $(n=1, m=2),(n=4, m=3)$. After the lost qubit is initialised with a fresh $\ket{0}$ qubit and projected into the computational basis of the $m\mathrm{th}$ qubit.

$$
\begin{aligned}
\ket{\psi}_r &= H_n I_{1,\cdots,4 \neq n} \ket{\psi}_r
\ket{\psi}_r &= \mathrm{CNOT}_{n,(1,\cdots,4 \neq n)}\ket{\psi}_r
\end{aligned}
$$

If $\ket{\psi}_r$ was initially projected into the $P_0$ then $\ket{\psi}_r == \ket{\psi}$ otherwise the following operation will recover $\ket{\psi}$

$$
\begin{aligned}
\ket{\psi}_r &= X_n I_{1,\cdots,4 \neq n} \ket{\psi}_r
    &= \ket{\psi}
\end{aligned}
$$

To implement the circuit on a known erasure without classically controlling the implementation of the $X$ gate. After the projection into the computational basis on the $m\mathrm{th}$ qubit a $\mathrm{CNOT}$ with the $m\mathrm{th}$ qubit as control and an ancilla as target would need to be implemented. After applying the Hadamard ($H_n$) and three consecutive $ \mathrm{CNOT}_{n,(1,\cdots,4 \neq n)}$ a $\mathrm{CNOT}$ with the ancilla as control and $n\mathrm{th}$ qubit as target would implement the appropriate correction.


## Question 4

### a)

$$
\begin{aligned}
\ket{\Psi} &= \frac{1}{2}(\ket{000} + \ket{010} + \ket{100} + \ket{111})\\
\end{aligned}
$$

The only stabiliser and  stabiliser generator for the state $\ket{0}$ is the Pauli operator $Z$.  However, the stabiliser for for the state $\ket{000}$ is $Z_{1}Z_{2}Z_{3}$ which is shorthand for $Z\otimes Z \otimes Z$. The stabiliser generators for the stabiliser $Z_{1}Z_{2}Z_{3}$ are $Z_{1}I_{2}I_{3}$, $I_{1}Z_{2}I_{3}$, $I_{1}I_{2}Z_{3}$. Working through the circuit for creating the state $\ket{\Psi}$ conjugating gates from left to right we calculate the the stabliser generators $S_i$ for $\ket{\Psi}$. First conjugating with the gate $H_1H_2I_3$ S the stabiliser generators becomes:

$$
\begin{aligned}
S_1 &= (H_{1}H_{2}I_{3})Z_{1}I_{2}I_{3} (H_{1}H_{2}I_{3})^{\dagger}\\
&= X_{1}I_{2}I_{3}\\
S_2 &= (H_{1}H_{2}I_{3})I_{1}Z_{2}I_{3} (H_{1}H_{2}I_{3})^{\dagger}\\
&= I_{1}X_{2}I_{3}\\
S_2 &= (H_{1}H_{2}I_{3})I_{1}I_{2}Z_{3} (H_{1}H_{2}I_{3})^{\dagger}\\
&= I_{1}I_{2}Z_{3}
\end{aligned}
$$

Than conjugating with the Toffoli gate ($\ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}$) the stabiliser S is:
$$
\begin{aligned}
S_1 &= \ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(X_{1}I_{2}I_{3})\ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}\\
    &=  \ket{0}_{1}\bra{0}I_{2}I_{3} (X_{1}I_{2}I_{3})\ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(X_{1}I_{2}I_{3})\ket{0}_{1}\bra{0}I_{2}I_{3}
    &= (\ket{0}_{1}\bra{1} + \ket{1}_{1}\bra{0})\mathrm{CNOT}_{2,3}\\
    &= X_1 \mathrm{CNOT}_{2,3}\\
S_2 &= \ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(I_{1}X_{2}I_{3})\ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}\\
    &=  \ket{0}_{1}\bra{0}I_{2}I_{3} (I_{1}X_{2}I_{3})\ket{0}_{1}\bra{0}I_{2}I_{3}  + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(I_{1}X_{2}I_{3}) \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}\\
    &=  \ket{0}_{1}\bra{0}X_{2}I_{3} + \ket{1}_{1}\bra{1}X_2X_3\\
    &= \mathrm{CNOT}_{1,3}X_2\\
S_2 &= \ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(I_{1}I_{2}Z_{3})\ket{0}_{1}\bra{0}I_{2}I_{3} + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}\\
    &=  \ket{0}_{1}\bra{0}I_{2}I_{3} (I_{1}I_{2}Z_{3})\ket{0}_{1}\bra{0}I_{2}I_{3}  + \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}(I_{1}I_{2}Z_{3}) \ket{1}_{1}\bra{1}\mathrm{CNOT}_{2,3}\\
    &= \ket{0}_{1}\bra{0}I_{2}Z_{3} + \ket{1}_{1}\bra{1}Z_2Z_3\\
    &=\mathrm{CPHASE}_{1,2} Z_3
\end{aligned}
$$

Therefore the stabiliser generators for $\ket{\psi}$ are $S_1 = X_1 \mathrm{CNOT}_{2,3}, S_2 = \mathrm{CNOT}_{1,3}X_2, S_3 = \mathrm{CPHASE}_{1,2} Z_3$.

### b)
The circuit for fault tolerantly preparing the state $\ket{\psi}$ through measurement of its stabilisers is created by adding 3 ancilla corresponding to the 3 stabilisers are prepared in the computational basis. Controlled stabiliser measurements are applied applied to the state. This gives the following circuit created in qiskit.

In [3]:
psi = QuantumRegister(3,name="psi")
anc = QuantumRegister(3,name="anc")
data = ClassicalRegister(3,name="anc_out")
toffoli_circuit = QuantumCircuit(psi,anc,data)
toffoli_circuit.h(psi[0:2])
toffoli_circuit.h(anc)
toffoli_circuit.toffoli(0,1,2)
toffoli_circuit.barrier()

stabiliser_meas_circuit = QuantumCircuit(psi,anc,data)
# stabiliser X_1CNOT_2,3
stabiliser_meas_circuit.cnot(anc[0],psi[2])
stabiliser_meas_circuit.toffoli(anc[0],psi[1],psi[0])

# stabiliser CNOT_1,3X_2
stabiliser_meas_circuit.cnot(anc[1],psi[1])
stabiliser_meas_circuit.toffoli(anc[1],psi[0],psi[2])
# stabiliser CNOT_1,3X_2
stabiliser_meas_circuit.cz(anc[2],psi[2])
stabiliser_meas_circuit.cz([anc[2],psi[0]],psi[1])

stabiliser_meas_circuit.barrier()
stabiliser_meas_circuit.h(anc)
stabiliser_meas_circuit.measure(anc,data)
stabiliser_meas_circuit.draw()

fault_tolerent_toffoli_circuit = toffoli_circuit.compose(stabiliser_meas_circuit)
fault_tolerent_toffoli_circuit.draw()

┌───┐      ░      ┌───┐                    ░               
    psi_0: ┤ H ├──■───░──────┤ X ├───────■─────────■──░───────────────
           ├───┤  │   ░      └─┬─┘┌───┐  │         │  ░               
    psi_1: ┤ H ├──■───░────────■──┤ X ├──┼──────■──■──░───────────────
           └───┘┌─┴─┐ ░ ┌───┐  │  └─┬─┘┌─┴─┐    │     ░               
    psi_2: ─────┤ X ├─░─┤ X ├──┼────┼──┤ X ├─■──┼─────░───────────────
           ┌───┐└───┘ ░ └─┬─┘  │    │  └─┬─┘ │  │     ░ ┌───┐┌─┐      
    anc_0: ┤ H ├──────░───■────■────┼────┼───┼──┼─────░─┤ H ├┤M├──────
           ├───┤      ░             │    │   │  │     ░ ├───┤└╥┘┌─┐   
    anc_1: ┤ H ├──────░─────────────■────■───┼──┼─────░─┤ H ├─╫─┤M├───
           ├───┤      ░                      │  │     ░ ├───┤ ║ └╥┘┌─┐
    anc_2: ┤ H ├──────░──────────────────────■──■─────░─┤ H ├─╫──╫─┤M├
           └───┘      ░                               ░ └───┘ ║  ║ └╥┘
anc_out: 3/═══════════════════════════════════════════════════╩══╩══╩═
                                                              0  1  2